<a href="https://colab.research.google.com/github/shashank3110/Master_Thesis_BA_DeepVis/blob/master/colab_notebooks/siamese_network_saliency_maps_gcam_gcam%2B%2B_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/LLNL/fastcam.git

!pip install pytorch_gradcam
!pip install Cython
!pip install torch

Cloning into 'fastcam'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 598 (delta 113), reused 130 (delta 70), pack-reused 414
Receiving objects: 100% (598/598), 18.54 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (353/353), done.
     |████████████████████████████████| 6.0MB 6.3MB/s 
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-cp36-none-any.whl size=5270 sha256=f543b8204bc6e8b87a4165ed41c3ef5db04c9e05642607473fdd1d040f3acd80
  Stored in directory: /root/.cache/pip/wheels/e8/1e/35/d24150a078a90ce0ad093586814d4665e945466baa89907300
Successfully built pytorch-gradcam


#**Compute Saliency,Gradcam,Gradcam++ modules**

In [ ]:
# -*- coding: utf-8 -*-
%cd fastcam

import warnings
import os
import argparse

import tensorflow as tf
import math
import numpy as np
import torch
import torchvision
import torch.utils.data
from torch.utils.data import DataLoader,TensorDataset
import logging
from keras.layers import Input
from keras.layers.merge import concatenate
import torch.nn as nn
from torch.nn import BatchNorm3d,Conv3d,ReLU,MaxPool3d,Linear,AdaptiveAvgPool3d,Flatten,Softmax
import torch.nn.functional as F

from datetime import datetime
from torch.utils import data
import time
import skimage.io as sio
import os
import shutil
import pandas as pd
from random import shuffle
from skimage.transform import resize
import skimage.io as sio
from scipy.io import savemat,loadmat
import cv2

import mask
import draw
import norm
import misc

from torchvision import models
from torchvision.utils import make_grid, save_image
from gradcam.utils import visualize_cam
from gradcam import GradCAMpp, GradCAM
from matplotlib import pyplot as plt



def get_smoe_map(x,relu=False):

  '''
  Scaled map order equivalent map computation fumction:
  reference from the saliency map paper : https://github.com/LLNL/fastcam.git and adapted/modified  to our usecase
  '''

  print(f' smoe input shape={x.shape}')
  if relu:
    x=tf.nn.relu(x).numpy()
  print(f'x range={np.amax(x),np.amin(x)}')
  
  m   = np.mean(x,axis=-1)+0.0000001 
  # m   = np.mean(x,axis=0)+0.0000001 #avoid log 0
  
  x   = x + 0.0000001
  # k   = np.log2(m) - np.mean(np.log2(x), axis=0)
  k   = np.log2(m) - np.mean(np.log2(x), axis=-1)
  print(f'log of mean={np.log2(m)}, mean of log={np.mean(np.log2(x), axis=-1)}')
  print(f'k={k}')
  k   = k + 0.0000001
  # k   = np.log10(m) - np.mean(np.log10(x), axis=-1)
  print(np.array_equal(np.zeros(k.shape),k))
  print(f'{x.shape,k.shape,np.amin(k)}')
  print(f'kmax, kmin={np.min(k),np.max(k)}')
  print(f'mean={m}')
  th  = k * m
  print(f'smoe map={th}')
  print(f'smoe output shape={th.shape}')
  return th

def get_std_map(x):
  '''
  STD based map alternative to SMOE.
  '''
  print(f'before std map shape ={x.shape}')
  m = np.std(x,axis=-1)

  print(f'std map shape ={m.shape}')

  return m

def get_norm(x,const_mean=None,const_std=None):
  '''
  get norm refrence from the saliency map paper : https://github.com/LLNL/fastcam.git and adapted/modified  to our usecase
  '''
  s0      = x.shape[0]
  s1      = x.shape[1]
  s2      = x.shape[2]

  # x       = np.reshape(x,(1,s0*s1))
  x       = np.reshape(x,(1,s1*s2))
  print(f'get norm func x after reshape={x.shape} ')

  '''
      Compute Mean
  '''
  if const_mean is None:
      m       = np.mean(x,axis=1)
      m       = np.reshape(m,(m.shape[0],1))
  else:
      m       = const_mean

  print(f'get norm func x after mean reshape={m.shape} ') 
  '''
      Compute Standard Deviation
  '''
  if const_std is None:
      s       = np.std(x,axis=1)
      s       = np.reshape(s,(s.shape[0],1))
  else:
      s       = const_std
  
  '''
      The normal cumulative distribution function is used to squash the values from within the range of 0 to 1
  '''

  s=torch.tensor(s)
  x       = 0.5*(1.0 + torch.erf((x-m)/(s*torch.sqrt(torch.tensor(2.0)))))
  print(x.shape)    
  # x       = x.reshape(1,s0,s1)
  x       = x.reshape(1,s1,s2)

  print(f'map after norm={x,x.shape}')
  return x


def combine_sal_maps(smaps,output_size,weights,map_num,resize_mode='bilinear',do_relu=False):

  '''
  Combined saliency maps are computed here .
  '''

  bn  = smaps[0].shape[0]
  cm  = torch.zeros((bn, 1, output_size[0], output_size[1]), dtype=smaps[0].dtype, device=smaps[0].device)
  ww  = []
  
  '''
      Now get each saliency map and resize it. Then store it and also create a combined saliency map.
  '''
  for i in range(len(smaps)):
      # assert torch.is_tensor(smaps[i]), "Each saliency map must be a Torch Tensor."
      wsz = smaps[i].shape
      w   = np.reshape(smaps[i],(wsz[0], 1, wsz[1], wsz[2]))#smaps[i].reshape(wsz[0], 1, wsz[1], wsz[2])
   
      w   = nn.functional.interpolate(w, size=output_size, mode=resize_mode, align_corners=False) 
      ww.append(w)        # should we weight the raw maps ... hmmm
      
      cm  += (w * weights[i])

  '''
      Finish the combined saliency map to make it a weighted average.
  '''
  weight_sum =sum(weights)
  cm  = cm / weight_sum
  cm  = cm.reshape(bn, output_size[0],output_size[1])
  
  ww  = torch.stack(ww,dim=1)
  ww  = ww.reshape(bn, map_num, output_size[0], output_size[1])
  

  
  return cm, ww

def compute_saliency_tf(base_path,inputs,tf_model):
  '''
   Saliency maps are computed for specicied layers and then combined
  '''

  # gender=inputs[1]
  # gender=tf.reshape(gender,[1,1])
  img=inputs
  img_chunk= img #tf.convert_to_tensor(img)
  print(img_chunk.shape)
  img_chunk = tf.reshape(img_chunk,[1,121,145,1])
  layers=[layer.name for layer in tf_model.layers]
  outputs=[]

 
  j=0
  for i,l in enumerate(layers):
    if 'activation' in l:
      val=tf_model.get_layer(name=l).output
      print(i,j,l,val.shape)
      j+=1
      outputs.append(val) 
  outputs.append(encoder.output) 
                                        
  test_tf_model=tf.keras.models.Model(tf_model.inputs, outputs)
 
  predictions = test_tf_model(img_chunk)

  # Specify or experiment  with layers we want to compute saliency maps for.

  # hooks=[predictions[0],predictions[1],predictions[2],predictions[8],predictions[14],predictions[20],predictions[23]\
  #        ,predictions[29],predictions[35],predictions[41],predictions[47],predictions[50],\
  #        predictions[56],predictions[62],predictions[65]]#predictions[:layer_end]
  # hooks= [predictions[0],predictions[2],predictions[17],predictions[47],predictions[62]] #1x1 and 3x3 cnn

  #these layers were picked as the outputs have diffrent scale dimensions,  we can experimentwith other layers as well.
  hooks=[predictions[0],predictions[9],predictions[22],predictions[41],predictions[51]] 


  # choose specific channels / filters
  for x in hooks:
    print('ouput shapes layerwise')
    print(x.shape)


  #smoe saliency maps
  # sal_maps       = [ get_norm(get_smoe_map(np.mean(x.numpy()[:,:,:,:],axis=-1))) for x in hooks ]
  sal_maps       = [ get_norm(get_smoe_map(x.numpy())) for x in hooks ]

  #std dev saliency maps
  # sal_maps       = [ get_norm(get_std_map(np.mean(x.numpy()[:,:,:,:,:],axis=-2))) for x in hooks ]


  for smaps in sal_maps:
    print(smaps.shape)
  
  # all layer scale maps with equal weightage
  weights=np.ones(len(hooks))
 
  map_num=len(hooks)

  f, axarr = plt.subplots(1,1,figsize=(10,10))
  raw= np.mean(img_chunk[0,:,:,:],axis=-1)
  raw= raw/np.max(raw)
  r=axarr.imshow(raw,cmap='jet')
  axarr.set_title('Input image')
  cbar=plt.colorbar(r,fraction=0.01, pad=0.04)
  cbar.set_clim(0,1)
  plt.savefig(base_path+'input_chunk.png')

  csal_maps,sal_maps = combine_sal_maps(sal_maps,output_size=[in_height,in_width],weights=weights,map_num=map_num)
  output_path = base_path +'Map_Combined.png'
  f, axarr = plt.subplots(1,1,figsize=(10,10))
  csal_map=csal_maps[0,:,:].numpy()
  imcs=csal_map/np.max(csal_map)
  im = axarr.imshow(imcs,cmap='jet')
  axarr.set_title('Combined saliency map')
  cbar=plt.colorbar(im,fraction=0.01, pad=0.04)
  cbar.set_clim(0,1)
  plt.savefig(output_path)

  il = [sal_maps[0,i,:,:] for i in range(map_num)] # Put each saliency map into the figure
  il.append(csal_maps[0,:,:])                       # add in the combined map at the end of the figure
  images        = [torch.stack(il, 0)]          
  images        = make_grid(images, nrow=5)
  sal_img=images.unsqueeze(1)
  output_path=base_path +'Sal_Maps.png'
  save_image(sal_img,output_path)

  input_path = output_path
  f, axarr = plt.subplots(1,1,figsize=(10,10))
  im=sio.imread(input_path)
  im=axarr.imshow(np.mean(im,axis=-1)/255, cmap='jet');
  axarr.set_title('layerwise saliency maps')
  cbar=plt.colorbar(im,fraction=0.01, pad=0.04)
  cbar.set_clim(0,1)
  output_path=base_path +'Sal_Maps_jet.png'
  plt.savefig(output_path)


  

  return csal_maps









def get_grads(layer_name,tf_model,inputs):
  '''
  computes gradients for GCAM/GCAM++
  '''

  cam_list=[]

  img=inputs
  grad_model = tf.keras.models.Model([tf_model.inputs], [tf_model.get_layer(name=layer_name).output, tf_model.output])
  
  img_chunk=img #tf.convert_to_tensor(img)
  img_chunk = tf.reshape(img_chunk,[1,121,145,1])
 
  with tf.GradientTape() as tape:
      conv_outputs, predictions = grad_model(img_chunk)
      print(f'predictions={predictions}')
      print(f'conv_outputs shape={conv_outputs.shape},predictions shape={predictions.shape}')
      loss=predictions[0]

     
  output = conv_outputs[0] 
  print(f'entering tape gradients')

  grads = tape.gradient(loss, conv_outputs)[0]
  print(type(grads))
  print(f'Crossed tape gradients')
  gate_f = tf.cast(output > 0, 'float32')
  gate_r = tf.cast(grads > 0, 'float32')
  # now there are 2 choice either use grads(raw grads) or use guided grads)
  guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

  print(f'Entering reduce mean using guided_grads with shape={guided_grads.shape}')
  #guided grads
  weights = tf.reduce_mean(guided_grads, axis=(0,1,2))


  print(f'Computing CAM using output with shape:{output.shape}')

  print(f'weights={weights.shape}')
  cam = np.zeros(output.shape[0:3], dtype=np.float32)
  print(cam.shape)

   
  cam=tf.reduce_sum(tf.multiply(output,weights),axis=-1)
  cam_list.append(cam)
  return cam_list,grads,loss,weights,output,img_chunk




def compute_gcam_and_gcam_pp(layer_name,model,inputs):
  '''
  Generates GCAM/GCAM++
  '''

  cam_list,grads,loss,weights,output,img_chunk = get_grads(layer_name,model,inputs)
  
  heatmap_list=[]
  for i,cam in enumerate(cam_list):#as we are doing chunk wise so this camlist will have only one cam
   
    print(f'cam shape={cam.shape}')
    
    #gcam
    cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))

    cam_map = np.maximum(cam_map,0)
    original_image=img_chunk.numpy()
   
    heatmap = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

  
    print(original_image.shape)
    image=np.mean(original_image[0,:,:,:],axis=-1)
    print(image.shape)

    mri_img=image#np.squeeze(image)
    heatmap_list.append(heatmap)
    # print(heatmap.min(),heatmap.max())

    heatmap_gcam = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())

      
    print(f'heatmap_gcam shape={heatmap_gcam.shape}')
    gcam_img=(np.mean(heatmap_gcam,axis=-1)* 255).astype("uint8")
   
    #gcam++
    print(f'grads shape ={grads.shape},tf.exp(loss) shape={tf.exp(loss).shape}')
    loss = np.mean(loss)
    conv_first_grad = tf.exp(loss)*grads
    #second_derivative
    conv_second_grad = tf.exp(loss)*grads*grads
    #triple_derivative
    conv_third_grad = tf.exp(loss)*grads*grads*grads
    global_sum = np.sum(tf.reshape(output,(-1,conv_first_grad.shape[2])), axis=0)
    print(f'conv_first_grad shape={conv_first_grad.shape},output.shape={output.shape},conv_second_grad shape={conv_second_grad.shape} ,  conv_third_grad shape={conv_third_grad.shape}, global_sum.shape={global_sum.shape}  ')

    alpha_num = conv_second_grad
  
    alpha_denom = conv_second_grad*2.0 + conv_third_grad*global_sum.reshape((1,1,conv_first_grad.shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

   

    alphas_thresholding = np.where(weights, alphas, 0.0)
    print(f'alphas_thresholding shape={alphas_thresholding.shape}')
    alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
    alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))
    
    print(f'alpha_normalization_constant_processed shape={alpha_normalization_constant_processed.shape}')
    
    alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad.shape[2]))
    weights_alpha=tf.reduce_sum(tf.multiply(weights,alphas),axis=0)
    
    cam=tf.reduce_sum(tf.multiply(output,weights_alpha),axis=-1)
    
    cam_map=resize(cam,(img_chunk.shape[1],img_chunk.shape[2],img_chunk.shape[3]))
  
    
    print(f'cam_map={cam_map.shape}')
    cam_map = np.maximum(cam_map, 0)

    heatmap_gcam_pp = (cam_map - cam_map.min()) / (cam_map.max() - cam_map.min())


    gcam_pp_img=(np.mean(heatmap_gcam_pp,axis=-1) * 255).astype("uint8")
    ##
    print(img_chunk.shape,mri_img.shape,gcam_img.shape,type(mri_img),type(gcam_img))
    
        
    return image, gcam_img,gcam_pp_img

def combine_sal_gcam(base_path,csmap,gcam_img,gcam_pp_img,image,layer_name ,angle=0,result_path='/content/drive/My Drive/BA_Estimation/final_results/'):
  '''
  3 kinds of map computed : saliency map , saliency map combined  with GCAM, saliency map combined with GCAM++ 
  '''
  print(gcam_img.shape,csmap.shape,gcam_pp_img.shape,image.shape)
  #saliency map

  raw_tensor=torch.from_numpy(image).unsqueeze(0)
  heatmap_csmap, result_csmap = visualize_cam(csmap, raw_tensor)
  getMask                 = mask.SaliencyMaskDropout(keep_percent = 0.1, scale_map=False)
  hard_masked_csmap,_       = getMask(raw_tensor.unsqueeze(0),csmap)#.squeeze(0))
  hard_masked_csmap        = hard_masked_csmap.squeeze(0)
  masked_csmap             = misc.AlphaMask(raw_tensor, csmap.squeeze(0)).squeeze(0)
  

  vmin=0
  vmax=1.0
  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map + gradcam')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map+gradcam with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_csmap,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_csmap[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'saliency_only_fig.png')

  print(hard_masked_csmap.permute([2,0,1]).shape)
  csmap_img = torch.mean(csmap,axis=0).numpy()
  output_path   = base_path+"csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':csmap_img ,'shape':csmap_img.shape})
  output_path   = base_path+"heatmap_csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_csmap.permute([1,2,0]).numpy() ,'shape':heatmap_csmap.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_csmap.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_csmap.permute([1,2,0]).numpy() ,'shape':result_csmap.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_csmap.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_csmap.permute([1,2,0]).numpy() ,'shape':hard_masked_csmap.permute([1,2,0]).shape})
  output_path   = base_path+"masked_csmap.mat" 
  savemat(output_path,{'data':masked_csmap.numpy() ,'shape':masked_csmap.numpy().shape})
  masked_csmap_mat = loadmat(output_path)['data']
  plt.clf()
  p=plt.imshow(masked_csmap_mat,cmap='jet')
  plt.colorbar(p)      
  plt.clim(0.8,1)
  output_path   = base_path+"masked_csmap_0.8.png" 
  plt.savefig(output_path)

  #gcam

  if np.max(gcam_img) ==0:
    gcam_img = gcam_img+0.0000001
  if np.max(gcam_pp_img) ==0:
    gcam_pp_img = gcam_pp_img+0.0000001
  gcam_img_tensor=torch.from_numpy(gcam_img).unsqueeze(0)
  mask_gcam = csmap*(gcam_img_tensor)
  mask_gcam=mask_gcam/mask_gcam.max()
  

  

  heatmap_gcam, result_gcam = visualize_cam(mask_gcam, raw_tensor)
  getMask                 = mask.SaliencyMaskDropout(keep_percent = 0.1, scale_map=False)
  hard_masked_gcam,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam)#.squeeze(0))
  hard_masked_gcam        = hard_masked_gcam.squeeze(0)
  masked_gcam             = misc.AlphaMask(raw_tensor, mask_gcam.squeeze(0)).squeeze(0)
  mx= str(np.max(masked_gcam.numpy()))
  plt.imsave(base_path+'masked_gcam_unnormalized_{0}max.png'.format(mx),masked_gcam.numpy(),cmap='jet')
  # masked_gcam              = misc.RangeNormalize(masked_gcam)


  #gcam++
  gcam_pp_img_tensor=torch.from_numpy(gcam_pp_img).unsqueeze(0)
  mask_gcam_pp = csmap*(gcam_pp_img_tensor)
  mask_gcam_pp=mask_gcam_pp/mask_gcam_pp.max()
  raw_tensor=torch.from_numpy(image).unsqueeze(0)
  heatmap_gcam_pp, result_gcam_pp = visualize_cam(mask_gcam_pp, raw_tensor)

  hard_masked_gcam_pp,_       = getMask(raw_tensor.unsqueeze(0),mask_gcam_pp)#.squeeze(0))
  hard_masked_gcam_pp         = hard_masked_gcam_pp.squeeze(0)
  masked_gcam_pp           = misc.AlphaMask(raw_tensor, mask_gcam_pp.squeeze(0)).squeeze(0)
  mx= str(np.max(masked_gcam_pp.numpy()))
  plt.imsave(base_path+'masked_gcam_pp_unnormalized_{0}max.png'.format(mx),masked_gcam_pp.numpy(),cmap='jet')
  # masked_gcam_pp           = misc.RangeNormalize(masked_gcam_pp) # avoid this step as it will normalize to 0 to 1 hence not good while comparing multiple scans


  

  #
  output_path   = base_path+"raw_img.mat"
  savemat(output_path,{'data':raw_tensor.numpy() ,'shape':raw_tensor.shape})
  background_img=loadmat(output_path)['data']

  base_path+='_'+layer_name

  #save gcam and gcam++ fig
  vmin=np.amin([np.min(gcam_img),np.min(gcam_pp_img)])
  vmax=np.amax([np.max(gcam_img),np.max(gcam_pp_img)])

  f, axarr = plt.subplots(1,2,figsize=(10,10))
  img_plot = axarr[0].imshow(gcam_img,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0].set_title('Gradcam')
  img_plot = axarr[1].imshow(gcam_pp_img,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1].set_title('Gradcam++')
  plt.colorbar(img_plot,fraction=0.046, pad=0.04)
  plt.savefig(base_path+'gcam_gcam++_fig.png')
  
  
  #gcam

  
  vmin=0
  vmax=1.0
  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map + gradcam')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_gcam,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map+gradcam with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_gcam,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_gcam[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'sal+gcam_fig.png')

  print(hard_masked_gcam.permute([2,0,1]).shape)
  output_path   = base_path+"gcam_img.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_img ,'shape':gcam_img.shape})
  output_path   = base_path+"heatmap_gcam.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam.permute([1,2,0]).numpy() ,'shape':heatmap_gcam.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_gcam.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam.permute([1,2,0]).numpy() ,'shape':result_gcam.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_gcam.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam.permute([1,2,0]).shape})
  output_path   = base_path+"masked_gcam.mat" 
  savemat(output_path,{'data':masked_gcam.numpy() ,'shape':masked_gcam.numpy().shape})
  masked_gcam_mat = loadmat(output_path)['data']
  plt.clf()
  p=plt.imshow(masked_gcam_mat,cmap='jet')
  plt.colorbar(p)      
  plt.clim(0.8,1)
  output_path   = base_path+"masked_gcam_0.8.png" 
  plt.savefig(output_path)
  
  #gcam_pp
  vmin=np.amin([torch.min(raw_tensor),torch.min(csmap),torch.min(heatmap_gcam_pp),torch.min(result_gcam_pp),torch.min(masked_gcam_pp),torch.min(hard_masked_gcam_pp)])
  vmax=np.amax([torch.max(raw_tensor),torch.max(csmap),torch.max(heatmap_gcam_pp),torch.max(result_gcam_pp),torch.max(masked_gcam_pp),torch.max(hard_masked_gcam_pp)])

  vmin=0
  vmax=1.0

  f, axarr = plt.subplots(2,3,figsize=(20,20))
  img_plot = axarr[0][0].imshow(torch.mean(raw_tensor,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][0].set_title('input')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][0])
  cbar.set_clim(0,1)
  img_plot = axarr[0][1].imshow(torch.mean(csmap,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][1].set_title('combined saliency map')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][1])
  cbar.set_clim(0,1)
  img_plot = axarr[0][2].imshow(torch.mean(heatmap_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[0][2].set_title('saliency map + gradcam++')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[0][2])
  cbar.set_clim(0,1)
  img_plot = axarr[1][0].imshow(torch.mean(result_gcam_pp,axis=0),vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][0].set_title('saliency map+gradcam++ with alpha blend')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][0])
  cbar.set_clim(0,1)
  img_plot = axarr[1][1].imshow(masked_gcam_pp,vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][1].set_title('mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][1])
  cbar.set_clim(0,1)
  img_plot = axarr[1][2].imshow(hard_masked_gcam_pp[0],vmin=vmin,vmax=vmax, cmap='jet');
  axarr[1][2].set_title('hard mask')
  cbar=plt.colorbar(img_plot,fraction=0.046, pad=0.04,ax=axarr[1][2])
  cbar.set_clim(0,1)
  plt.savefig(base_path+'sal+gcam++_fig.png')


  raw_img = torch.mean(raw_tensor,axis=0).numpy()
  output_path   = base_path+"raw_input.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':raw_img ,'shape':raw_img.shape})

  f, axarr = plt.subplots(1,1,figsize=(10,10))
  
  r=axarr.imshow(raw_img,cmap='gray')
  axarr.set_title('raw gray image')
  cbar=plt.colorbar(r,fraction=0.046, pad=0.04)
  cbar.set_clim(0,1)
  plt.savefig(base_path+'raw_gray_cbar.png')

  
  output_path   = base_path+"gcam_pp_img.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':gcam_pp_img ,'shape':gcam_pp_img.shape})
  output_path   = base_path+"heatmap_gcam_pp.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':heatmap_gcam_pp.permute([1,2,0]).numpy() ,'shape':heatmap_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"result_gcam_pp.png"
  savemat(output_path.split('.png')[0] +'.mat',{'data':result_gcam_pp.permute([1,2,0]).numpy() ,'shape':result_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"hard_masked_gcam_pp.png" 
  savemat(output_path.split('.png')[0] +'.mat',{'data':hard_masked_gcam_pp.permute([1,2,0]).numpy() ,'shape':hard_masked_gcam_pp.permute([1,2,0]).numpy().shape})
  output_path   = base_path+"masked_gcam_pp.mat" 
  savemat(output_path,{'data':masked_gcam_pp.numpy() ,'shape':masked_gcam_pp.numpy().shape})
  masked_gcam_pp_mat = loadmat(output_path)['data']
  plt.clf()
  p=plt.imshow(masked_gcam_pp_mat,cmap='jet')
  plt.colorbar(p)      
  plt.clim(0.8,1)
  output_path   = base_path+"masked_gcam_pp_0.8.png" 
  plt.savefig(output_path)
  

  im=ndimage.rotate(masked_gcam_pp_mat,angle)
  max=1
  im = im/max #optional step as in our case max is 1 also use any contant val.

  im[im<0.3]=np.nan
  plt.imshow(im,cmap='jet')
  plt.axis('off')
  plt.clim(0,1)
  plt.savefig(result_path+'_result.png')

  ##############################################################################
  '''
  These are the images used in the final results the ones inside result_path
  '''
  plt.clf() # clear existing figure
  #mask overlaid on gray matter
  print(f'background shape before={background_img.shape}')
  im2=background_img[0]
  print(f'background shape after={im2.shape}')
  im2=ndimage.rotate(im2,angle)
  im2=1-im2
  gray=plt.imshow(im2,cmap='gray')
  plt.axis('off')
  im=ndimage.rotate(masked_gcam_pp_mat,angle)
  im[im<0.3]=np.nan
  heat=plt.imshow(im,cmap='jet')
  plt.axis('off')
  plt.clim(0,1)
  plt.colorbar()
  plt.savefig(result_path+'_result_overlay.png')

  ##############################################################################

  ## Saliency only
  masked_csmap=masked_csmap.numpy()

  frac=0.5

  t='masked_only_saliency'
  max =  1 #np.amax(masked_gcam)
  r1=(masked_csmap/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.3
  r1=(masked_csmap/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.8
  r1=(masked_csmap/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')


  ## GCAM
  masked_gcam=masked_gcam.numpy()

  frac=0.5

  t='masked_gcam'
  max =  1 #np.amax(masked_gcam)
  r1=(masked_gcam/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.3
  r1=(masked_gcam/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.8
  r1=(masked_gcam/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  ## GCAM++
  ## fraction mask  map for overlaying GCAM++
  masked_gcam_pp=masked_gcam_pp.numpy()

  frac=0.5

  t='masked_gcam_pp'
  max = 1 #np.amax(masked_gcam_pp)
  r1=(masked_gcam_pp/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.3
  r1=(masked_gcam_pp/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  frac=0.8
  r1=(masked_gcam_pp/max)

  r1[np.where(r1<frac*np.max(r1))]=0
  # r1[np.where(r1<np.median(r1))]=0
  plt.imsave(base_path+'nodiff_{0}_{1}.png'.format(frac,t),r1,cmap='jet')

  ##



/content/fastcam


# **Tf records loading and parsing utility**

In [ ]:
def get_selected_scan_from_subjects(data_path,subject_ids,label_df,selected_scans):
    '''
    Same as get_scan_from_subjects in case we need specific scan path and not all scans of a patient.
    '''
    scans=[]
    labels=[]
    gender=[]
    cdr=[]
    ids=[]
    subject_ids = set(subject_ids)
    for subject in subject_ids :
        path=os.path.join(data_path,subject)
        paths=os.listdir(path)

        ids.extend([scan_id.split('.')[0] for scan_id in paths  if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        scans.extend([ os.path.join(path,scan_id) for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        
    
        labels.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['Age'].to_list()[0] for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans ])
        gender.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['M/F'].to_list()[0] for scan_id in paths   if scan_id.split('/')[-1].split('.')[0] in selected_scans])
        cdr.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['CDR'].to_list()[0] for scan_id in paths  if scan_id.split('/')[-1].split('.')[0] in selected_scans])

    return scans,labels,gender,ids,cdr

def get_scan_from_subjects(data_path,subject_ids,label_df):
    '''
    Scan paths,cdr,gender,labels  of subjects .
    '''
    scans=[]
    labels=[]
    gender=[]
    cdr=[]
    ids=[]
    subject_ids = set(subject_ids)
    for subject in subject_ids :
        path=os.path.join(data_path,subject)
        paths=os.listdir(path)
        ids.extend([scan_id.split('.')[0] for scan_id in paths ])
        scans.extend([ os.path.join(path,scan_path) for scan_path in paths  ])
 
        labels.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['Age'].to_list()[0] for scan_id in paths   ])
        gender.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['M/F'].to_list()[0] for scan_id in paths  ])
        cdr.extend([label_df[label_df['MRI ID']==scan_id.split('.')[0]]['CDR'].to_list()[0] for scan_id in paths ])
    #print(labels,gender)
           
    # shuffle(scans)
    
    return scans,labels,gender,ids,cdr


def get_test_files(label_path,data_path,debug_mode_subject=None,selected_scans=[]):
    '''
    Primary function to get scans.
    '''

    data = pd.read_csv(label_path)
    data = data.rename(columns={'MR ID':'MRI ID'})
    data['M/F'] = encode_gender(data)
    if debug_mode_subject is None:
      test_ids = os.listdir(data_path)
    else:
      test_ids=debug_mode_subject
    
    shuffle(test_ids)
    if len(selected_scans)>0:
      test_patients,test_labels,test_gender,scan_ids,test_cdr = get_selected_scan_from_subjects(data_path,test_ids,data,selected_scans)
    else:
      test_patients,test_labels,test_gender,scan_ids,test_cdr = get_scan_from_subjects(data_path,test_ids,data)
   
    return test_patients,scan_ids, test_labels,test_gender,test_cdr
   
def encode_gender(data):
    '''
    Categorical encoding. for gender.
    '''
    data['M/F'] = pd.Categorical(data['M/F'])
    
    return data['M/F'].cat.codes

def parse_function_image(example_proto):

    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_shape': tf.io.FixedLenFeature([], tf.string)
    }

    content = tf.io.parse_single_example(example_proto, features=features)

    content['image_shape'] = tf.io.decode_raw(content['image_shape'], tf.int32)
    content['image'] = tf.io.decode_raw(content['image'], tf.float32)
    content['image'] = tf.reshape(content['image'], content['image_shape'])

    return content['image']

# **Create Siamese model encoder (reference: Nivedita. V)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf
import pickle

from tensorflow.keras import backend as K

from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply, add, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import l2


latent_dim = 128
  
  
def conv2d_bn(x,
              filters,
              strides,
              padding='same'
              ):

    x = tf.keras.layers.Conv2D(filters, (3, 3),
        strides=strides,
        padding=padding, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4),
        use_bias=False)(x)

    bn_axis = -1
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, scale=False)(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x
    

def conv2d_bn_1x1(x,
              filters,
              strides,
              padding='same'
              ):

    x = tf.keras.layers.Conv2D(filters, (1, 1),
        strides=strides,
        padding=padding, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4),
        use_bias=False)(x)

    bn_axis = -1
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, scale=False)(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x


def conv2d_bn_7x7(x,
              filters,
              strides=(2, 2),
              padding='same'
              ):
			  
    x = tf.keras.layers.Conv2D(filters, (7, 7),
        strides=strides,
        padding=padding, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4),
        use_bias=False)(x)

    bn_axis = -1
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, scale=False)(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x
	
	
def residual_module1(layer_in, n_filters):
    # conv1
    x = conv2d_bn(layer_in, n_filters, strides=(1, 1), padding='same')

    # conv2
    conv2 = conv2d_bn(x, n_filters, strides=(1, 1), padding='same')

    # add filters, assumes filters/channels last
    layer_out = add([conv2, layer_in])
    # activation function
    layer_out = Activation('relu')(layer_out)

    return layer_out


def residual_module2(layer_in, n_filters):
    # conv1
    x = conv2d_bn(layer_in, n_filters, strides=(2, 2), padding='same')
    # conv2
    conv2 = conv2d_bn(x, n_filters, strides=(1, 1), padding='same')
    
    #projection shortcut for mismatch in number of channels
    y = conv2d_bn_1x1(layer_in, n_filters, strides=(2, 2), padding='same')

    # add filters, assumes filters/channels last
    layer_out = add([conv2, y])
    # activation function
    layer_out = Activation('relu')(layer_out)

    return layer_out
	
	
def res34(inp):
    channel_axis = -1
    #Instantiates the Inflated 3D Inception v1 architecture.

    # Downsampling via convolution (spatial and temporal)
    x = conv2d_bn_7x7(inp, 64, strides=(2, 2), padding='same')
    x = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same')(x)
	
    x = residual_module1(x,64)
    x = residual_module1(x,64)
    x = residual_module1(x,64)

    x = residual_module2(x,128)
    x = residual_module1(x,128)
    x = residual_module1(x,128)
    x = residual_module1(x,128)

    x = residual_module2(x,256)
    x = residual_module1(x,256)
    x = residual_module1(x,256)
    x = residual_module1(x,256)
    x = residual_module1(x,256)
    x = residual_module1(x,256)

    x = residual_module2(x,512)
    x = residual_module1(x,512)
    x = residual_module1(x,512)
	
    x = tf.keras.layers.GlobalAveragePooling2D(name = 'gap')(x)
	
    # FCN with Relu activation, kernel_regularizer=l2(1e-3)
    x = tf.keras.layers.Dense(units=512, kernel_regularizer=l2(1e-3), name='dense1')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    # FCN with Relu activation, kernel_regularizer=l2(1e-3)
    x = tf.keras.layers.Dense(units=256, kernel_regularizer=l2(1e-3), name='dense2')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    z = tf.keras.layers.Dense(units = latent_dim)(x)
    z = tf.keras.layers.Activation('relu')(z)
    z = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x, axis=1))(z)

    return z
	
def model_create():
    # defining input shape
    input = tf.keras.Input(shape=(121, 145, 1))

    # base for encoder
    z = res34(input)

    # defining model for encoder
    encoder = Model(inputs = input, outputs = z, name='encoder')
    print(encoder.summary())

    return encoder

# **Assigning Siamese model's encoder's weights to encoder created from above cell**

In [ ]:

# type(siam_model.get_layer(index=3).weights)
def assign_encoder_weights(siam_model,encoder):
  encoder_weights= siam_model.get_layer(index=3).weights
  j=0
  for i,l in enumerate(encoder.layers[1:-2]):
    # print(l)
    # if i > 0 and i< 150:
    if 'activation' in l.name or 'pool' in l.name or 'add' in l.name or 'gap' in l.name or 'dropout' in l.name:
      # j+=1
      continue
    
    elif 'batch' in l.name:
      print(f'I,J={i,j}')
      print(encoder.layers[i+1].name,encoder_weights[j].name)

      beta=encoder_weights[j].numpy()
      mean=encoder_weights[j+1].numpy()
      var=encoder_weights[j+2].numpy()

      encoder.layers[i+1].set_weights([beta,mean,var])
      j+=3
 
    elif 'dense' in l.name:
      print(f'I,J={i,j}')
      print(encoder.layers[i+1].name,encoder_weights[j].name)
      w=encoder_weights[j].numpy()
      b=encoder_weights[j+1].numpy()
      encoder.layers[i+1].set_weights([w,b])
      j+=2
    else:
      print(f'I,J={i,j}')
      print(encoder.layers[i+1].name,encoder_weights[j].name)
     
      encoder.layers[i+1].set_weights([encoder_weights[j].numpy()])
      j+=1

  return encoder
  # print(l.weights)
    


#**Compute Visualizations for siamese model  on testset** 

In [ ]:
cf={'Pretrained_Model':{'path':'/content/drive/My Drive/BA_Estimation/models/exp_siam/Model'},'Paths':\
      {'labels':'/content/drive/My Drive/BA_Estimation/csv_data/oasis1_oasis3_labels.csv',\
       'test_tfrecord':'/content/drive/My Drive/BA_Estimation/tf_records_data/training_testing_exp4'}}# #all_cdr

In [ ]:

siam_model =  tf.python.keras.models.load_model(cf['Pretrained_Model']['path'],compile=False)
encoder=model_create()
tf_model = assign_encoder_weights(siam_model,encoder)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: Res34_model is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 121, 145, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 61, 73, 64)   3136        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 61, 73, 64)   192         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 61, 73, 64)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [ ]:

print(tf.__version__)


dt_string = datetime.now().strftime('%d-%m-%Y-%H-%M')+'_smoe_maps_blockend_scale_endlayers_equal_weights'

label_path= cf['Paths']['labels'] 
data_path= cf['Paths']['test_tfrecord'] 
# exp='exp_siam_ad'
exp='exp_siam_healthy'
# debug_mode_subject= ['OAS31054']#
# debug_mode_subject = ['OAS30884','OAS30720','OAS30344','OAS30541'] #OAS30911
# debug_mode_subject=['OAS30440','OAS30445','OAS31140','OAS30952','OAS30199','OAS30344']#exp4
# debug_mode_subject=['OAS31165','OAS30819','OAS30392','OAS30310','OAS31035'] #exp2
# debug_mode_subject = ['OAS30736','OAS30282','OAS30622','OAS30190','OAS30440','OAS30826','OAS30286','OAS30987','OAS30583','OAS30070','OAS30952']
debug_mode_subject=['OAS10104','OAS10416','OAS10045','OAS31035','OAS30310','OAS30392','OAS30956','OAS30956','OAS30093','OAS30008']
# debug_mode_subject=None
test_patients,scan_ids, test_labels,test_gender,test_cdr = get_test_files(label_path,data_path,debug_mode_subject)
tfr=tf.data.TFRecordDataset(test_patients)
img_tf=tfr.map(map_func=lambda a:parse_function_image(a))

gender_dict={0:'Female',1:'Male'}
for i,im in enumerate(img_tf): 
  print(type(im),im.shape)
  img=im.numpy()
  print(img.shape)

  max_intensity=0
  csmap_list=[]
  for chunk_id in [36,42,48,54,60]:#82,88,94]:#range(1,21):

    start = (chunk_id)
    end = start+1
    
    img_chunk=torch.tensor(img[:,:,start]).unsqueeze(0)
    img_chunk = img_chunk.unsqueeze(-1)
    
    print(f'input shape={img_chunk.shape}')

    input_tensor = img_chunk
    #***********

    in_height   = input_tensor.size()[1]
    in_width    = input_tensor.size()[2]
    print(test_gender[i],input_tensor.shape,scan_ids[i])
 
    a=51
    n=35
    # layer_name='conv3d_'+str(n-1)
    layer_name='activation_'+str(a)
    # conv_path ='conv1_'+str(n)+'_tfw'
    # conv_path ='conv1_'+str(n)+'_blockend_scale_endlayers_equal_weights'
    conv_path ='conv1_'+str(n)
    

    #axial
    base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_axial/'+dt_string\
    +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
    result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string +'/cdr'+cdr_val+'/axial/'+str(chunk_id)+'/'
    if not os.path.exists(result_path):
      os.makedirs(result_path)
    result_path += scan_ids[i]
    csmap_a=compute_saliency_tf(base_path,inputs=input_tensor,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,input_tensor)
    combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_a,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=-270,result_path=result_path ) 
    

    #sagittal
    base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_sagittal/'+dt_string\
    +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
    result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string +'/cdr'+cdr_val+'/sagittal/'+str(chunk_id)+'/'
    if not os.path.exists(result_path):
      os.makedirs(result_path)
    result_path += scan_ids[i]
    img_s= torch.from_numpy(img[start:end,:,:]).permute(2,1,0)
    csmap_s=compute_saliency_tf(base_path,inputs=img_s,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,img_s)
    combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_s,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=180,result_path=result_path ) 

    #coronal
    base_dir= '/content/drive/My Drive/BA_Estimation/results/sal_map_coronal/'+dt_string\
    +'_'+exp+'/'+scan_ids[i]+'_cdr'+str(test_cdr[i])
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+scan_ids[i]+'_chunk_'+str(chunk_id)
    result_path = '/content/drive/My Drive/BA_Estimation/final_results/{0}/'.format(case)+dt_string +'/cdr'+cdr_val+'/coronal/'+str(chunk_id)+'/'
    if not os.path.exists(result_path):
      os.makedirs(result_path)
    result_path += scan_ids[i]
    img_c= torch.from_numpy(img[:,start:end,:]).permute(2,0,1)
    img_c=img_c.unsqueeze(0)
    img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,1), mode='nearest') 
    csmap_c=compute_saliency_tf(base_path,inputs=img_c,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,img_c)
    combine_sal_gcam(path+scan_ids[i]+'_cdr'+str(test_cdr[i])+'_'+gender_dict[test_gender[i]],csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name,angle=180,result_path=result_path ) 

  


# **Get healthy and ad subjects from eval set and compute mean images at cdr and gender level**

In [ ]:
# np.zeros(1)
import copy
exp='exp_siam'
healthy_path = '/content/drive/My Drive/BA_Estimation/models/{0}/Shuffled_labels_evalhealthy.csv'.format(exp)
ad_path = '/content/drive/My Drive/BA_Estimation/models/{0}/Shuffled_labels_evalAD.csv'.format(exp)
healthy_path = '/content/drive/My Drive/BA_Estimation/models/{0}/testset.csv'.format(exp)
ad_path = '/content/drive/My Drive/BA_Estimation/models/{0}/outlier.csv'.format(exp)
hdf = pd.read_csv(healthy_path)
adf = pd.read_csv(ad_path)
print(hdf.columns)

sub =  hdf['patient_id'].values.tolist()
sub.extend(adf['patient_id'].values.tolist())
sub= list(set(sub))
scans= copy.deepcopy(sub)
for i,s in enumerate(sub) :
  if s.startswith('OAS1'):
    s= s[:9] #OAS1_0123_MR1 take first 9 characters
    sub[i] = s.replace('_','')
  elif s.startswith('OAS3'): #OAS31098_MR_d7178 #take just subject id
    sub[i] = s.split('_')[0]


In [ ]:
#compute mean slices according to gender and cdr on entire dataset
from collections import defaultdict
if __name__=='__main__':
  gender_dict={0:'Female',1:'Male'}
  gender_cdr_chunk_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
  gender_cdr_full_dict = defaultdict(lambda: defaultdict(list))
  dt_string = datetime.now().strftime('%d-%m-%Y-%H-%M')+'mean_scans_smoe_maps_blockend_scale_endlayers_equal_weights'

  label_path= cf['Paths']['labels']
  data_path= cf['Paths']['test_tfrecord']
  
  test_patients,scan_ids, test_labels,test_gender,test_cdr = get_test_files(label_path,data_path,debug_mode_subject=sub,selected_scans=scans)
  
  tfr=tf.data.TFRecordDataset(test_patients)
  img_tf=tfr.map(map_func=lambda a:parse_function_image(a))
  for i,im in enumerate(img_tf):
    print(i+1,im.shape)
    img=im.numpy()
    gender_cdr_full_dict[test_gender[i]][test_cdr[i]].append(img)
    

  



In [ ]:

slice_id =48
for g in gender_cdr_full_dict.keys():
          print(f'Gender = {gender_dict[g]}')
          for cdr,_ in gender_cdr_full_dict[g].items():
              print(f'CDR = {cdr}')
              full_imgs = np.array(gender_cdr_full_dict[g][cdr])
              full_mean = np.mean(full_imgs,axis=0)
              print(full_imgs.shape,full_mean.shape)
              matpath =  '/content/drive/My Drive/BA_Estimation/means/{0}/'.format(exp)
              if not os.path.exists(matpath):
                os.makedirs(matpath)
              savemat(matpath+'full_img_mean_cdr'+str(cdr)+'_'+gender_dict[g]+'.mat',{'data': full_mean,'shape':full_mean.shape})
              
              print('********************************')
              img_a = full_imgs[:,:,:,48]
              img_a = np.expand_dims(img_a,-1)
              slice_mean = np.mean(img_a,axis=0)
              savemat(matpath+'slice'+str(slice_id)+'axial_img_mean_cdr'+str(cdr)+'_'+gender_dict[g]+'.mat',{'data': full_mean,'shape':full_mean.shape})
              print(slice_mean.shape)
              
              img_s = np.mean(full_imgs[:,48,:,:],axis=0)
              img_s=np.expand_dims(img_s,0)
              slice_mean = torch.from_numpy(img_s).permute(2,1,0).numpy()
              print(slice_mean.shape)
              savemat(matpath+'slice'+str(slice_id)+'sagittal_img_mean_cdr'+str(cdr)+'_'+gender_dict[g]+'.mat',{'data': full_mean,'shape':full_mean.shape})
              
              img_c = np.mean(full_imgs[:,:,48,:],axis=0)
              # img_c=np.expand_dims(img_c,0)
              img_c= torch.from_numpy(img_c).permute(1,0)

              img_c=img_c.unsqueeze(-1)
              img_c=img_c.unsqueeze(0)
              img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,1), mode='nearest') 
              slice_mean = img_c[0,0,:,:,:].numpy()
              print(slice_mean.shape)
              savemat(matpath+'slice'+str(slice_id)+'coronal_img_mean_cdr'+str(cdr)+'_'+gender_dict[g]+'.mat',{'data': full_mean,'shape':full_mean.shape})
             

Gender = Female
CDR = 0.0


#**Compute Visualizations for Mean  subjects test set**

The mean was computed on the test set ateach cdr value for male and female separately and then visualized .

In [ ]:
mat_path = '/content/drive/My Drive/BA_Estimation/means/exp_siam/full/'
files = os.listdir(mat_path)
files

['full_img_mean_cdr1.0_Male.mat',
 'full_img_mean_cdr0.0_Male.mat',
 'full_img_mean_cdr0.5_Female.mat',
 'full_img_mean_cdr0.5_Male.mat',
 'full_img_mean_cdr1.0_Female.mat',
 'full_img_mean_cdr2.0_Female.mat',
 'full_img_mean_cdr0.0_Female.mat']

In [ ]:
# files[0].split('_')[-1].split('.')[0], files[1].split('_')[-2][-3:]

('Male', '0.0')

In [ ]:
mat_path = '/content/drive/My Drive/BA_Estimation/means/exp_siam/full/'
files = os.listdir(mat_path)

# dt_string = datetime.now().strftime('%d-%m-%Y-%H-%M')+'_smoe_maps_blockend_scale_endlayers_equal_weights'

label_path= cf['Paths']['labels'] 
data_path= cf['Paths']['test_tfrecord'] 
# exp='exp_siam_ad'
exp='exp_siam_means'


gender_dict={0:'Female',1:'Male'}
reverse_gender_dict = {'Female':0,'Male':0}
for i,f in enumerate(files): #OAS30686_d0030
  # print(type(f),im.shape)

  test_gender = f.split('_')[-1].split('.')[0]
  gender_tensor= np.array(reverse_gender_dict[test_gender])
  test_cdr = f.split('_')[-2][-3:]
  img=loadmat(mat_path+f)['data']
  csmap_list=[]
  for chunk_id in [48]:

    start = (chunk_id)
    end = start+1
    

    img_chunk=torch.tensor(img[:,:,start]).unsqueeze(0)
    img_chunk = img_chunk.unsqueeze(-1)
    
    print(f'input shape={img_chunk.shape}')
   
    input_tensor = img_chunk

    in_height   = input_tensor.size()[1]
    in_width    = input_tensor.size()[2]
    print(test_gender,input_tensor.shape,f)

    a=51
    n=35

    layer_name='activation_'+str(a)

    conv_path ='conv1_'+str(n)


    #axial
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string+ '/sal_map_axial/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    csmap_a=compute_saliency_tf(base_path,inputs=input_tensor,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,input_tensor)
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_a,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 


    #sagittal
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string+'/sal_map_sagittal/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    img_s= torch.from_numpy(img[start:end,:,:]).permute(2,1,0)
    csmap_s=compute_saliency_tf(base_path,inputs=img_s,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,img_s)
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_s,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 

    #coronal
    base_dir= '/content/drive/My Drive/BA_Estimation/means/'+dt_string + '/sal_map_coronal/' \
    +exp+'/'+f+'_cdr'+str(test_cdr)
    path = base_dir+'/'+str(chunk_id)+'_'+conv_path+'/'
    print(path)
    if not os.path.exists(path):
      os.makedirs(path)
    base_path  = path+f+'_chunk_'+str(chunk_id)
    img_c= torch.from_numpy(img[:,start:end,:]).permute(2,0,1)
    img_c=img_c.unsqueeze(0)
    img_c = torch.nn.functional.upsample(img_c.unsqueeze(0), size=(121,145,1), mode='nearest') 
    csmap_c=compute_saliency_tf(base_path,inputs=img_c,tf_model=tf_model)
    image,gcam_img,gcam_pp_img = compute_gcam_and_gcam_pp(layer_name,tf_model,img_c)
    combine_sal_gcam(path+f+'_cdr'+str(test_cdr)+'_'+test_gender,csmap_c,gcam_img,gcam_pp_img,image,layer_name=layer_name ) 


    